<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Applying Fourier Transform on signal series.</b>
    <br>
<b style = 'font-size:24px;font-family:Arial;color:#E37C4D'>Discrete Fast Fourier Transform and Convolve functions </b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>Fourier transform</b> is one of the most important algorithms in all of 
science, mathematics, and communications technology.
<br>
<p style = 'font-size:16px;font-family:Arial'> The Fourier Theorem states that any signal can be perfectly represented as a sum of sine waves, each sine wave having its own phase, frequency, and amplitude. This provides an efficient way to <b>transform a signal (e.g. a radio wave) from the time domain to the frequency domain</b>. The time domain is what we are used to seeing (e.g. a radio wave is a fluctuation of signal amplitude over time). The frequency domain shows how much energy is in the signal at each frequency (sine wave speed). This allows us to view the individual components of a signal, and therefore provides a better understanding of complicated signals.

<br>

<p style = 'font-size:16px;font-family:Arial'>There are four parts to a wave:
wavelength(λ), period(T), frequency(f), and amplitude (A)</p>
<img id="signal wave" src="images/signal.png" alt="signal wave" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The Angular Frequency is ω = 2π × f</p>

### Applications for FFT are
<p style = 'font-size:16px;font-family:Arial'>
<ul>
            <li  style = 'font-size:16px;font-family:Arial'>Digital Signal Processing</li>
            <li style = 'font-size:16px;font-family:Arial'>Noise reduction in signals</li>
            <li  style = 'font-size:16px;font-family:Arial'>Seasonality detection in timeseries</li>
            <li  style = 'font-size:16px;font-family:Arial'>Image Processing</li>
        </ul>    
</p>

<p style = 'font-size:16px;font-family:Arial'>
    In this demonstration we will use new <b>UAF </b>(Unbounded Array Framework) Vantage capabilities. With UAF functionality, Vantage can process one-dimensional series (series indexed by time or space) and two-dimensional arrays (matrix indexed by time, space or time and space). The series can reside in any Teradata supported or Teradata accessible table or in an analytic result table (ART). We will use the below functions in this demonstration:
<ul>    
    <li style = 'font-size:16px;font-family:Arial'> <b>TD_GENSERIES4FORMULA</b>-provides the ability for you to define and apply a formula to generate a time series </li>
    <li style = 'font-size:16px;font-family:Arial'> <b>TD_DFFT & TD_IDFFT</b> - TD_DFFT Applies the Discrete Fast Fourier Transform to a one-dimensional series and returns a result series containing the computed Fourier Coefficients. The coefficients can be in rectangular (real, imaginary) or polar (amplitude, phase). TD_IDFFT reverses the effects of a Fourier transform on a one-dimensional series. </li>
    <li style = 'font-size:16px;font-family:Arial'> <b>TD_CONVOLVE</b> - Applies digital filter to one-dimensional series by convolving time series with kernel filter </li>
    <li style = 'font-size:16px;font-family:Arial'> <b>TD_PLOT</b> - provides the ability to generate charts. The generated charts can be in the JPG, PNG, or SVG formats </li>
</ul>    
</p>  


<p style = 'font-size:16px;font-family:Arial;'>Let's do a simple example on sample data on how the signals with random noise looks like and how do we recognize noise using FFT.</p>
<p style = 'font-size:16px;font-family:Arial'><b>Steps</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Connect to Vantage</li>
    <li>Create data points for the series(genData)</li>
    <li>Create signal series (SINUSOIDS4) with 4 frequencies using TD_GENSERIES4FORMULA</li>
    <li>Create signal series (SINUSOIDS4_noise) with 4 frequencies and random noise added</li>
    <li>Apply TD_DFFT Fourier Transform function on both the series so find the significant frequencies</li>
    <li>Apply TD_IDFFT function to see that we can regenerate the original series</li>
    <li>Preparing a signal frequency of 100Hz to use as a filter</li>
    <li>Apply the TD_CONVOLVE function to the series (SINUSOIDS4_noise) to cancel all the frequencies except of 100Hz</li>
    <li>Clean up</li>
</ul>    
<hr>

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Connect to Vantage</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Let's start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
Set query_band='DEMO=FourierTransform.ipynb;' update for session;

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
call space_report();  -- optional, takes about 10 seconds

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Create the dataset</b>
<p style = 'font-size:16px;font-family:Arial'>We will create two signals with 400 datapoints. One pure signal which will have 4 sinusoidal frequencies of 2, 25, 50 & 100 Hz and other which will have random noise added as well. 

In [ ]:
/* This table will be create 400 data points */
CREATE TABLE genData(ID integer, row_i integer, MAGNITUDE float);

In [ ]:
INSERT INTO genData values(1, 1, 1.0);
INSERT INTO genData SELECT ID, row_i+1, MAGNITUDE+1.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+2, MAGNITUDE+2.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+4, MAGNITUDE+4.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+8, MAGNITUDE+8.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+16, MAGNITUDE+16.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+32, MAGNITUDE+32.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+64, MAGNITUDE+64.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+128, MAGNITUDE+128.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+256, MAGNITUDE+256.0 FROM genData;

In [ ]:
--To get only 400 data points
DELETE FROM GenData WHERE ROW_I>400.0;

In [ ]:
/*************************************************************************/
/*  Generate 400 Samplepoint Series which is the sum of 4 Sinusoids      */
/*  Sinusoids have frequencies (k) of: 2, 25, 50, 100                    */
/*************************************************************************/
EXECUTE FUNCTION INTO ART(SINUSOIDS4)
TD_GENSERIES4FORMULA(
  SERIES_SPEC(TABLE_NAME(GenData), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
  PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))
  ),
  FUNC_PARAMS(Formula('Y = 4.0*sin(6.283185307179586476*X1/200.0) +
                           4.0*sin(6.283185307179586476*X1/16.0 + 3.14159265358979323/4.0) + 
                           4.0*sin(6.283185307179586476*X1/8.0 + 3.14159265358979323/2.0) +
                           4.0*sin(6.283185307179586476*X1/4.0 + 3.14159265358979323/8.0)'))
);

/* 4.0*sin(6.283185307179586476*X1/200.0) - 2Hz frequnecy
   4.0*sin(6.283185307179586476*X1/16.0 + 3.14159265358979323/4.0) - 25Hz with phase added    
   4.0*sin(6.283185307179586476*X1/8.0 + 3.14159265358979323/2.0) - 50Hz with phase added 
   4.0*sin(6.283185307179586476*X1/4.0 + 3.14159265358979323/8.0) - 100Hz with phase added 
*/

<p style = 'font-size:16px;font-family:Arial'>Let's see what data is generated for the series.

In [ ]:
select top 100 * from SINUSOIDS4;

<p style = 'font-size:16px;font-family:Arial'>Table SINUSOIDS4 is the table containing signal data of 4 sinusidal waves.</p>
<p style = 'font-size:16px;font-family:Arial'>We can see how the signal looks like using TD_PLOT.</p>


In [ ]:
/**********************************************************************************/
/*                              PLOT generated sinusoidal signal                  */
/**********************************************************************************/

EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(SINUSOIDS4), ROW_AXIS(SEQUENCE(ROW_I)), 
                SERIES_ID(ID),
                PAYLOAD(FIELDS(Magnitude), CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line'))], IMAGE('png')));

<p style = 'font-size:16px;font-family:Arial'><i>* Please <b> right click on the IMAGE column </b> from the output and choose view image to see the plot generated. </i>

<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above to View Image,  you should have seen image as below

<img id="sine wave" src="images/sin4wave.png" alt="Sine wave with no noise added" width="400" />
<p style = 'font-size:16px;font-family:Arial'>Sine wave with no noise added.</p>

<p style = 'font-size:16px;font-family:Arial'>Now, let us add some random <b>noise </b>in the signal. We have added a noise component 3* random(1,10)/random(1,3) in same sine frequencies as above signal.

In [ ]:
EXECUTE FUNCTION INTO ART(SINUSOIDS4_noise)
TD_GENSERIES4FORMULA(
  SERIES_SPEC(TABLE_NAME(GenData), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
  PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))
  ),
  FUNC_PARAMS(Formula('Y = 4.0*sin(6.283185307179586476*X1/200.0) + 
                           4.0*sin(6.283185307179586476*X1/16.0 + 3.14159265358979323/4.0) + 
                           4.0*sin(6.283185307179586476*X1/8.0 + 3.14159265358979323/2.0) + 
                           4.0*sin(6.283185307179586476*X1/4.0 + 3.14159265358979323/8.0) + 
                           (3* random(1,10)/random(1,3))'))
);

<p style = 'font-size:16px;font-family:Arial'>Let's see how the signal looks like now when it has noise added.

In [ ]:
/**********************************************************************************/
/*             PLOT generated sinusoidal signal with noise added                  */
/**********************************************************************************/

EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(SINUSOIDS4_noise), ROW_AXIS(SEQUENCE(ROW_I)), 
                SERIES_ID(ID),
                PAYLOAD(FIELDS(Magnitude), CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line'))], IMAGE('png')));


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. TD_DFFT and TD_IDFFT</b>
<br>
<p style = font-size:16px;font-family:Arial>Discrete Fourier Transform (DFFT) & Inverse DFFT are the functions available in Vantage Clearscape Analytics which can convert any timeseries in frequency domain and vice versa using the Vantage's UAF (Unbounded Array Framework)</p>
<p style = 'font-size:16px;font-family:Arial'>Let us plot Fourier Transform to see if we can distinguish the main signals (sinusoids)
</p>

In [ ]:
/**********************************************************************************/
/*          Applying TD_DFFT function with signal with no noise                   */
/**********************************************************************************/
EXECUTE FUNCTION INTO VOLATILE ART(DfftRaw_INTEGRAL) 
TD_DFFT(
  SERIES_SPEC( TABLE_NAME(SINUSOIDS4), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
               PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL)) ),
  FUNC_PARAMS( FREQ_STYLE("K_INTEGRAL"), HUMAN_READABLE(1) ),
  OUTPUT_FMT( CONTENT(AMPL_PHASE_RADIANS) ) );

<p style = 'font-size:16px;font-family:Arial'>The output of the TD_DFFT function creates a series with identifiers (seriesID & RowID) corresponding to the original series and Fourier Amplitude & Phase coefficient corresponding to ROW_I index (or Real & Imaginary depending on the parameters provided in the function).

In [ ]:
/* Return Fourier Coeff rows having a significant magnitude */
SELECT * FROM DfftRaw_INTEGRAL WHERE Amplitude_Magnitude > 1.0;

In [ ]:
/**********************************************************************************/
/*          Applying TD_DFFT function with signal with noise                      */
/**********************************************************************************/
EXECUTE FUNCTION INTO VOLATILE ART(DfftRaw_noise) 
TD_DFFT(
  SERIES_SPEC( TABLE_NAME(SINUSOIDS4_noise), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
               PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL)) ),
  FUNC_PARAMS( FREQ_STYLE("K_INTEGRAL"), HUMAN_READABLE(1) ),
  OUTPUT_FMT( CONTENT(AMPL_PHASE_RADIANS) ) );

<p style = 'font-size:16px;font-family:Arial'>Let's plot these transformed series to see if we can identify the significant frequencies in the signals.

In [ ]:
/******************************************************************************************/
/*                           Plot the results of the DFFT of the Signal                  */
/******************************************************************************************/

EXECUTE FUNCTION
TD_Plot
(   SERIES_SPEC(TABLE_NAME(DfftRaw_INTEGRAL), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(ID),
                PAYLOAD(FIELDS(Amplitude_Magnitude),CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line') )], IMAGE('png'))
);


<p style = 'font-size:16px;font-family:Arial'>In the image plot produced we can clearly see that the significant frequencies(2, 20, 50 & 100 Hz) in the signal which corresponds to the ones we used to create our signal.

In [ ]:
/******************************************************************************************/
/*                           Plot the results of the DFFT of the Signal                  */
/******************************************************************************************/
EXECUTE FUNCTION
TD_Plot
(   SERIES_SPEC(TABLE_NAME(DfftRaw_noise), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(ID),
                PAYLOAD(FIELDS(Amplitude_Magnitude),CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line') )], IMAGE('png'))
);

<p style = 'font-size:16px;font-family:Arial'>Now we can clearly see that both the series have the same significant frequencies but one has random noises which has created fluctuations in the DFFT plot above.

<p style = 'font-size:16px;font-family:Arial'><b>We can recreate our original signals(time domain) from the dfft (frequency domain)</b></p>

In [ ]:
/**********************************************************************************/
/*               TAKE INVERSE DFFT of PREVIOUS DFFT OUTPUT                        */
/**********************************************************************************/
EXECUTE FUNCTION INTO VOLATILE ART(ReturnOriginal) 
        TD_IDFFT(ART_SPEC(TABLE_NAME(DfftRaw_INTEGRAL),
                          PAYLOAD( FIELDS(AMPLITUDE_MAGNITUDE,PHASE_MAGNITUDE), 
                                   CONTENT(AMPL_PHASE_RADIANS))) );

In [ ]:
/******************************************************************************************/
/*                           Plot the results of the IDFFT of the Signal                  */
/******************************************************************************************/
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(ReturnOriginal), ROW_AXIS(SEQUENCE(ROW_I)), SERIES_ID(ID),
                PAYLOAD(FIELDS(Real_Amplitude_Magnitude), CONTENT(REAL))),
    FUNC_PARAMS(PLOTS[(TYPE('line'))], IMAGE('png')));

In [ ]:
EXECUTE FUNCTION INTO VOLATILE ART(ReturnOriginal_noise) 
        TD_IDFFT(ART_SPEC(TABLE_NAME(DfftRaw_noise),
                          PAYLOAD( FIELDS(AMPLITUDE_MAGNITUDE,PHASE_MAGNITUDE), 
                                   CONTENT(AMPL_PHASE_RADIANS))) );

In [ ]:
/******************************************************************************************/
/*                           Plot the results of the IDFFT of the Signal                  */
/******************************************************************************************/
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(ReturnOriginal_noise), ROW_AXIS(SEQUENCE(ROW_I)), SERIES_ID(ID),
                PAYLOAD(FIELDS(Real_Amplitude_Magnitude), CONTENT(REAL))),
    FUNC_PARAMS(PLOTS[(TYPE('line'))], IMAGE('png')));

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Signal Filtering</b>
<p style = 'font-size:16px;font-family:Arial'>We can use signal filtering to filter out the pure or significant frequencies from our signals. This can be used in noise filtering also. From our random noise signal, lets filter all the frequencies except 100hz.<br><br>
In Vantage Clearscape Analytics we have <b>TD_CONVOLVE</b> function which can filter one signal from another. We also have <b>TD_BINARYSERIESOP</b> function which can perform a point-wise mathematical operation (addition, subtraction, multiplication, or division) on two time series of equal size.<br> First we create a sin wave of 100hz.
</p>

In [ ]:
 /* creating filter series with 400 data points*/
 EXECUTE FUNCTION INTO ART(SINUSOIDS4_fil)
 TD_GENSERIES4FORMULA(
  SERIES_SPEC(TABLE_NAME(GenData), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
  PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))
  ),
  FUNC_PARAMS(Formula('Y = 4.0*sin(6.283185307179586476*X1/4.0)'))
);

<p style = 'font-size:16px;font-family:Arial'>Let's see how the filter signal looks like.

In [ ]:
/**********************************************************************************/
/*                              PLOT generated sinusoidal signal                  */
/**********************************************************************************/

EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(SINUSOIDS4_fil), ROW_AXIS(SEQUENCE(ROW_I)), 
                SERIES_ID(ID),
                PAYLOAD(FIELDS(Magnitude), CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line'))], IMAGE('png')));

<p style = 'font-size:16px;font-family:Arial'>Now, let's apply this series as the filter

In [ ]:
/*************************************************************************/
/*      Convolve Filter Kernel with the Original Signal                 */
/* Original Signal contained sinusoids at: 2, 25,50, 100HZ and noise     */
/*        Filter should erase everything except 100HZ                  */
/*************************************************************************/

EXECUTE FUNCTION INTO VOLATILE ART(ApplyLowPass) 
TD_CONVOLVE(SERIES_SPEC(TABLE_NAME(SINUSOIDS4_noise), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
                        PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))),
            SERIES_SPEC(TABLE_NAME(SINUSOIDS4_fil), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
                        PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))),
            INPUT_FMT(INPUT_MODE(MATCH)));

<p style = 'font-size:16px;font-family:Arial'>The TD_CONVOLVE function takes one series representing the source series to be filtered and the other series being the
filter kernel itself. The resultant output is a source series with the kernel filter applied.

In [ ]:
SHOW TABLE ApplyLowPass;

In [ ]:
/*************************************************************************/
/* Verify that the filtered Signal has the desired freq characteristics  */
/*                             using the DFFT                            */
/*************************************************************************/
--DROP TABLE DfftFilteredSins4;
EXECUTE FUNCTION INTO VOLATILE ART(DfftFilteredSins4) 
TD_DFFT(SERIES_SPEC(TABLE_NAME(ApplyLowPass), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
                    PAYLOAD( FIELDS(REAL_MAGNITUDE), CONTENT(REAL)) ),
        FUNC_PARAMS( FREQ_STYLE("K_INTEGRAL"), HUMAN_READABLE(1) ),
  OUTPUT_FMT( CONTENT(AMPL_PHASE_RADIANS) ) );


In [ ]:
EXECUTE FUNCTION
TD_Plot
(   SERIES_SPEC(TABLE_NAME(DfftFilteredSins4), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(ID),
                PAYLOAD(FIELDS(Amplitude_real_Magnitude),CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line') )], IMAGE('png'))
);

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Clean up</b>

In [ ]:
Drop TABLE genData;

In [ ]:
DROP TABLE SINUSOIDS4;

In [ ]:
DROP TABLE SINUSOIDS4_noise;

In [ ]:
Drop TABLE SINUSOIDS4_fil;

<p style = 'font-size:16px;font-family:Arial'>In this demo we have seen how we can transform timeseries to frequency domain to get significant frequencies and vice-versa and how we can filter out the frequencies from our signal series.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li> Unbounded Array Framework: <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework'>https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework</a></li>


<footer style="paddinga:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>